# Caffe学习系列：图像数据转换成db（leveldb&lmdb）文件

在深度学习的实际应用中，我们经常用到的原始数据是图片文件，如jpg，jpeg，png，gif等格式的，而且有可能图片的大小还不一致。而在Caffe中经常使用的数据类型是lmdb或leveldb，因此就产生一个问题：如何从原始图片文件转换成caffe中能够运行的db（leveldb/lmdb）文件？

在caffe中，作者为我们提供了这样一个文件：convert_imageset.cpp，存放在根目录下的tools文件夹下。编译之后，生成对应的可执行文件放在build/tools/下面，这个文件的作用就是用于将图片文件转换成caffe框架中能直接使用的db文件。

该文件的使用格式：
```
convert_imageset [FLAGS] ROOTFOLDERS/ LISTFILE DB_NAME
```
需要带四个参数：
- FLAGS：图片参数组，后面详细介绍
- ROOTFOLDERS：图片存放的绝对路径，从Linux系统根目录开始
- LISTFILE：图片文件列表清单，一般为一个txt文件，一行一张图片
- DB_NAME：最终生成的db文件存放目录

如果图片已经下载到本地电脑上了，那么我们首先需要创建一个图片列表清单，保存为txt。

本文以caffe程序中自带的图片为例，进行讲解，图片目录是example/images/，两张图片，一张为cat.jpg，另一张为fish_bike.jpg，表示两个类别。

我们创建一个sh脚本文件，调用Linux命令来生成图片清单：
```
sudo vim examples/images/create_filelist.sh
```
编辑这个文件，输入下面的代码并保存：
```
#!/usr/bin/env sh
DATA=examples/images
echo "Create train.txt ..."
rm -rf $DATA/train.txt
find $DATA -name *cat.jpg | cut -d '/' -f3 | sed "s/$/ 1/">>$DATA/train.txt
find $DATA -name *bike.jpg | cut -d '/' -f3 | sed "s/$/ 2/">>$DATA/tmp.txt
cat $DATA/tmp.txt>>$DATA/train.txt
rm -rf $DATA/tmp.txt
echo "Done..."
```

这个脚本文件中，用到了rm，find，cut，sed，cat等Linux命令。
- rm：删除文件
- find：寻找文件
- cut：截取路径
- sed：在每行的最后面加上标注。本例中将找到的\*cat.jpg文件加入标注为1，找到的\*bike.jpg文件加入标注为2
- cat：将两个类别合并在一个文件里

最终生成如下的一个train.txt文件：
```
cat.jpg 1
fish-bike.jpg 2
```

当然，图片很少的时候，手动编写这个列表清单文件就行了。但图片很多的情况，就需要用脚本文件来自动生成了。在以后的实际应用中，还需要生成相应的val.txt和test.txt文件，方法是一样的。

生成的这个train.txt文件，就可以作为第三个参数，直接使用了。

接下来，我们来了解一下FLAGS这个参数组，有些什么内容：
- -gray：是否以灰度图的方式打开图片。程序调用OpenCV库中的imread()函数来打开图片，默认为false
- -shuffle：是否随机打乱图片顺序。默认为false
- -backend：需要转换成的db文件格式，可选为leveldb或lmdb，默认为lmdb
- -resize_width/resize_height：改变图片的大小。在运行中，要求所有图片的尺寸一致，因此需要改变图片大小。程序调用OpenCV库的resize()函数来对图片放大缩小，默认为0，不改变
- -check_size：检查所有的数据是否有相同的尺寸。默认为false，不检查
- -encoded：是否将原图片编码放入最终的数据中，默认为false
- -encode_type：与前一个参数对应，将图片编码为哪一个格式：'png'，'jpg'，……

好了，知道这些参数后，我们就可以调用命令来生成最终的lmdb格式数据了。

由于参数比较多，因此我们可以编写一个sh脚本来执行命令：

首先，创建sh脚本文件：
```
sudo vim examples/images/create_lmdb.sh
```

编辑，输入下面的代码并保存：
```
#!/usr/bin/env sh
DATA=examples/images
rm -rf $DATA/img_train_lmdb
build/tools/convert_imageset --shuffle \
--resize_height=256 --resize_width=256 \
/home/xxx/caffe/examples/images/ $DATA/train.txt $DATA/img_train_lmdb
```
设置参数-shuffle，打乱图片顺序。设置参数-resize_height和-resize_width将所有图片尺寸都变为256×256。/home/xxx/caffe/examples/images/为图片保存的绝对路径。

最后，运行这个脚本文件：
```
sudo sh examples/images/create_lmdb.sh
```
就会在examples/images/目录下生成一个名为img_train_lmdb的文件夹，里面的文件就是我们需要的db文件了。